# Task

Handwritten digit recognition.

Given an image:
  (gray scale, 28 by 28, 1 channel)
Design an algorithm to detect the number.


# DATA

The MNIST database contains 60,000 training images and 10,000 testing images.
Each image is similar to   (gray scale, 28 by 28, 1 channel).

In addition, you have the label for each image in the training dataset and testing dataset.

Training dataset: training images+ label for each training image
Testing dataset: testing images+ label for each testing image


In [2]:
import numpy as np
from urllib import request
import gzip
import pickle

filename = [
["training_images","train-images-idx3-ubyte.gz"],
["test_images","t10k-images-idx3-ubyte.gz"],
["training_labels","train-labels-idx1-ubyte.gz"],
["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

def init():
    download_mnist()
    save_mnist()
#    print ((load()[0]).shape)
def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

if __name__ == '__main__':
    init()

Download complete.
Save complete.


In [3]:
import numpy as np
from urllib import request
import gzip
import math
import pickle


def grad_softmax_crossentropy(X, y):
    m = y.shape[0]
    ones_for_answers = np.zeros_like(X)
    ones_for_answers[np.arange(len(X)), y] = 1

    p = np.exp(X) / np.exp(X).sum(axis=-1, keepdims=True)
    return (- ones_for_answers + p) / m

def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)

        training_images, training_labels, testing_images, testing_labels = mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]
        # Normalize the images
        training_images.astype('float32')
        testing_images.astype('float32')
        training_images = training_images / 255
        testing_images = testing_images / 255
        return training_images, training_labels, testing_images, testing_labels

#============NOTE!!!!====================
'''========================
TRimg: training images
TRlab: label for training images
TSimg: test images
TSlab: label for test images
========================'''
TRimg,TRlab,TSimg,TSlab=load() #obtain the data

#============NOTE!!!!====================
'''========================
Try the following to know the data
========================'''

print(len(TRimg),len(TRlab),len(TSimg),len(TSlab))
print(len(TRimg[0]),len(TRlab),len(TSimg[0]),len(TSlab))
print(TRlab[10]) #test the label fot the 11th  item
arr_2d = np. reshape(TRimg[0], (28, 28))

60000 60000 10000 10000
784 60000 784 10000
3


# Model

# Training

In [9]:
#implementthe model
#parameters: W: weight matrix
# b: bias
"""
Assume your input is X
Your output is Y

For a two layer FC NN, two set of parameters:
W: weight matrix for layer 1
b: Bias for layer 1
W2: weight matrix for layer 2
b2: Bias for layer 2

"""
D=784 # input size
h=100 # hidden layer
K=10  # output size
#learning rate
step_size=0.1
#weight for regularization
reg=0.001
W=0.01*np.random.randn(D,h)
b=np.zeros((1,h))
W2=0.01*np.random.randn(h,K)
b2=np.zeros((1,K))

#mini batch setting
Epoc=10
BatchSize=32

for i in range(Epoc):
  for j in range(0,60000,32):
    X=TRimg[j:j+BatchSize]
    Y=TRlab[j:j+BatchSize]
    num_examples=X.shape[0]



    # forward 

    hidden_layer=np.maximum(0,np.dot(X,W)+b)
    scores=np.dot(hidden_layer,W2)+b2
    exp_scores=np.exp(scores)
    probs=exp_scores/np.sum(exp_scores,axis=1,keepdims=True)

    # Calculate the loss
    correct_logprobs=-np.log(probs[range(num_examples),Y])
    data_loss=np.sum(correct_logprobs)/num_examples
    reg_loss=0.5*reg*np.sum(W*W)+0.5*reg*np.sum(W2*W2)
    loss=data_loss+reg_loss

    #backpropagation (softmax+ crossentropy)
    dscores=probs
    dscores[range(num_examples),Y]-=1
    dscores/=num_examples

    #Direvative for the second layer (dw2 and db2)
    dw2=np.dot(hidden_layer.T,dscores) # direvative for the weight
    db2=np.sum(dscores, axis=0,keepdims=True)  # direvative for the bias
    dhidden=np.dot(dscores,W2.T)
    print(dhidden.shape)
    print(W2.T.shape)
    dhidden[hidden_layer <=0]=0 #Derivitative from Relu
    #Direvative for the first layer (dw and db)
    dw= np.dot(X.T,dhidden)
    db=np.sum(dhidden,axis=0,keepdims=True)

    #derivative from regularization
    print(W2.shape)
    dw2+=reg*W2
    dw+=reg*W

    #update the parameters
    W+=-step_size*dw
    b+=-step_size*db
    W2+=-step_size*dw2
    b2+=-step_size*db2  

(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)
(10, 100)
(100, 10)
(32, 100)


KeyboardInterrupt: ignored

In [ ]:
#use Test dataset for testing
# TSimg and Tslab
hidden_layer=np.maximum(0,np.dot(TSimg,W)+b) #Relu(x*w+b) first layer
scores=np.dot(hidden_layer,W2)+b2 #(hidden*w2+b2)
predicted_class=np.argmax(scores,axis=1) #softmax()
print("training accuracy: %.2f" % (np.mean(predicted_class==TSlab)))

training accuracy: 0.97


# Pytorch

In [ ]:
from __future__ import print_function

import numpy as np
from urllib import request
import gzip
import math
import pickle
from time import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 100)
        self.l2 = nn.Linear(100, 10)


    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.softmax(self.l2(x))

        return x

In [ ]:
model = Net()

#criterion = nn.CrossEntropyLoss()

#criterion = F.cross_entropy()
optimizer = optim.SGD(model.parameters(), lr=0.01)

epoch=10
BatchSize=32
for i in range(epoch):
  for j in range(0,60000,BatchSize):
    X=TRimg[j:j+BatchSize]
    X=torch.tensor(X)
    Y=TRlab[j:j+BatchSize]
    Y=torch.tensor(Y)
    num_examples=X.shape[0]
    optimizer.zero_grad()
    output = model(X.float())
    loss=F.cross_entropy(output, Y)
    #loss = criterion(output, Y)
    loss.backward()
    optimizer.step()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


# Test

In [ ]:
X=torch.tensor(TSimg)
Y=TSlab

output = model(X.float())
Pred=output.detach().numpy()
predicted_class=np.argmax(Pred,axis=1) #softmax()
print(predicted_class)
print(TSlab)
print("training accuracy: %.2f" % (np.mean(predicted_class==TSlab)))

[7 2 1 ... 4 8 6]
[7 2 1 ... 4 5 6]
training accuracy: 0.85


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
